In [19]:
import numpy as np

def expon(mean): 
    return np.random.exponential(scale=mean)

NEXT_EVENT_TYPE = 0
NUM_CUSTS_DELAYED = 0
NUM_DELAYS_REQUIRED = 1_000
NUM_EVENTS = 3
NUM_IN_Q = 0
SERVER_STATUS = 0

AREA_NUM_IN_Q = 0
AREA_NUM_IN_SYSTEM = 0
AREA_SERVER_STATUS = 0
MEAN_INTERARRIVAL = 0
MEAN_SERVICE = 0
SIM_TIME = 0
TIME_ARRIVAL = []
TIME_LAST_EVENT = 0
TIME_NEXT_EVENT = [0,0]
TOTAL_NUM_CUSTS = 0
TOTAL_OF_DELAYS = 0
TOTAL_SERVICE_TIME = 0
MAX_SYSTEM_TIME = 0
MAX_DELAY_IN_Q = 0
MAX_Q_LENGTH = 0
NUM_CUSTS_LONG_DELAY = 0

MEAN_INTERARRIVAL = 1.0
MEAN_SERVICE = 0.50

from enum import auto, Enum

class ServerStatus(Enum):
    IDLE = 0
    BUSY = 1


def initialize():
    """Initialize simulation variables"""
    # init sim clock
    global SIM_TIME
    SIM_TIME = 0.0
    
    # init state vars
    global SERVER_STATUS, NUM_IN_Q, TIME_LAST_EVENT
    SERVER_STATUS = ServerStatus.IDLE
    NUM_IN_Q = 0
    TIME_LAST_EVENT = 0.0

    # init stat counters
    global NUM_CUSTS_DELAYED, TOTAL_OF_DELAYS, AREA_NUM_IN_Q, AREA_SERVER_STATUS, TOTAL_NUM_CUSTS, MAX_SYSTEM_TIME
    NUM_CUSTS_DELAYED = 0
    TOTAL_OF_DELAYS = 0.0
    TOTAL_SERVICE_TIME = 0.0
    AREA_NUM_IN_Q = 0.0
    AREA_SERVER_STATUS = 0.0
    AREA_NUM_IN_SYSTEM = 0.0
    TOTAL_NUM_CUSTS = 0
    MAX_SYSTEM_TIME = 0.0
    MAX_DELAY_IN_Q = 0.0
    MAX_Q_LENGTH = 0
    NUM_CUSTS_LONG_DELAY = 0

    # init event list. No customers are present, so departure event is impossible
    global TIME_NEXT_EVENT, MEAN_INTERARRIVAL
    TIME_NEXT_EVENT = [
        SIM_TIME + expon(MEAN_INTERARRIVAL), # arrival event
        np.inf, # departure event
        480, # terminate sim event
    ]
    


def timing():
    """Compare the timings of the next of each possible event type to find
    the next event. Set NEXT_EVENT_TYPE accordingly, and advance the sim
    clock to the time of occurrent of this event.
    """
    global NEXT_EVENT_TYPE, NUM_EVENTS, TIME_NEXT_EVENT, SIM_TIME

    min_time_next_event = np.inf

    for i in range(NUM_EVENTS):
        if TIME_NEXT_EVENT[i] < min_time_next_event:
            min_time_next_event = TIME_NEXT_EVENT[i]
            NEXT_EVENT_TYPE = i

    SIM_TIME = TIME_NEXT_EVENT[NEXT_EVENT_TYPE]

    # if TIME_NEXT_EVENT[0] < TIME_NEXT_EVENT[1]:
    #     # arrival is next
    #     NEXT_EVENT_TYPE = 0
    #     SIM_TIME = TIME_NEXT_EVENT[0]
    # else:
    #     NEXT_EVENT_TYPE = 1
    #     SIM_TIME = TIME_NEXT_EVENT[1]
    


def arrive():
    """Handle the arrival of a new customer.
    If server is busy, add customer to queue. If server is idle, customer is
    immediately serviced with a delay of 0.
    """
    # print('arriving')
    delay = 0
    global TOTAL_NUM_CUSTS
    TOTAL_NUM_CUSTS += 1
    # schedule the next arrival
    global TIME_NEXT_EVENT, SIM_TIME, MEAN_INTERARRIVAL
    TIME_NEXT_EVENT[0] = SIM_TIME + expon(MEAN_INTERARRIVAL)

    global SERVER_STATUS
    if SERVER_STATUS is ServerStatus.BUSY:
        # if busy, go to the queue
        global NUM_IN_Q
        NUM_IN_Q += 1
        global MAX_Q_LENGTH
        if NUM_IN_Q > MAX_Q_LENGTH:
            MAX_Q_LENGTH = NUM_IN_Q

        # check for queue overflow
        if NUM_IN_Q > 100:
            raise Exception("Queue has overflowed!")
        
        # store arrival time of arriving customer at the end of the queue
        global TIME_ARRIVAL
        TIME_ARRIVAL += [SIM_TIME]
    else:
        # if idle, go directly to service
        delay = 0.0
        global TOTAL_OF_DELAYS, NUM_CUSTS_DELAYED
        TOTAL_OF_DELAYS += delay

        # increment number of customers delayed, make server busy
        NUM_CUSTS_DELAYED += 1
        SERVER_STATUS = ServerStatus.BUSY

        # schedule a departure
        global MEAN_SERVICE
        TIME_NEXT_EVENT[1] = SIM_TIME + expon(MEAN_SERVICE)


def depart():
    """Handle the departure of a customer from service."""
    # print('departing')

    global NUM_IN_Q
    if NUM_IN_Q == 0:
        # queue is empty, so server becomes idle and departures are impossible
        global SERVER_STATUS, TIME_NEXT_EVENT
        SERVER_STATUS = ServerStatus.IDLE
        TIME_NEXT_EVENT[1] = np.inf
    else:
        # there are customers in the queue
        NUM_IN_Q -= 1

        # compute delay of the customer who is starting service
        global SIM_TIME, TIME_ARRIVAL, TOTAL_OF_DELAYS
        delay = SIM_TIME - TIME_ARRIVAL[0]
        TOTAL_OF_DELAYS += delay
        global MAX_DELAY_IN_Q, NUM_CUSTS_LONG_DELAY
        if delay > MAX_DELAY_IN_Q:
            MAX_DELAY_IN_Q = delay
        if delay > 1:
            NUM_CUSTS_LONG_DELAY += 1

        # increment number of customers delayed, schedule next departure
        global NUM_CUSTS_DELAYED, MEAN_SERVICE
        NUM_CUSTS_DELAYED += 1
        TIME_NEXT_EVENT[1] = SIM_TIME + expon(MEAN_SERVICE)
        # record service time for the customer entering service
        global TOTAL_SERVICE_TIME, MAX_SYSTEM_TIME
        service_time = TIME_NEXT_EVENT[1] - SIM_TIME
        TOTAL_SERVICE_TIME += service_time
        total_system_time = delay + service_time
        if total_system_time > MAX_SYSTEM_TIME:
            MAX_SYSTEM_TIME = total_system_time

        # move customers up in queue
        TIME_ARRIVAL = TIME_ARRIVAL[1:]

def report():
    """Log statistics"""
    global TOTAL_OF_DELAYS, TOTAL_SERVICE_TIME, NUM_CUSTS_DELAYED, AREA_NUM_IN_Q, AREA_NUM_IN_SYSTEM, SIM_TIME, AREA_SERVER_STATUS, MAX_SYSTEM_TIME, MAX_DELAY_IN_Q, MAX_Q_LENGTH, NUM_CUSTS_LONG_DELAY
    print(f"\n\nAverage delay in queue {TOTAL_OF_DELAYS / NUM_CUSTS_DELAYED:11.3f} minutes\n\n")
    print(f"Maximum delay in queue {MAX_DELAY_IN_Q:10.3f}\n\n")
    print(f"Proportion of customers with delay > 1 {NUM_CUSTS_LONG_DELAY / TOTAL_NUM_CUSTS:10.3f}\n\n")
    print(f"Average total time in system {(TOTAL_OF_DELAYS + TOTAL_SERVICE_TIME) / TOTAL_NUM_CUSTS:11.3f} minutes\n\n")
    print(f"Maximum time in system {MAX_SYSTEM_TIME:10.3f}\n\n")
    print(f"Average number in queue {AREA_NUM_IN_Q / SIM_TIME:10.3f}\n\n")
    print(f"Maximum number in queue {MAX_Q_LENGTH:10d}\n\n")
    print(f"Average number in system {AREA_NUM_IN_SYSTEM / SIM_TIME:10.3f}\n\n")
    print(f"Server utilization {AREA_SERVER_STATUS / SIM_TIME:15.3f}\n\n")
    print(f"Time simulation ended {SIM_TIME:12.3f} minutes")
    

def update_time_avg_stats():
    """Prior to processing each event, this function updates the areas tracked
    for the calculation of continuous-time statistics."""
    global SIM_TIME, TIME_LAST_EVENT
    time_since_last_event = SIM_TIME - TIME_LAST_EVENT
    TIME_LAST_EVENT = SIM_TIME # current time

    # update area under num-in-queue function
    global AREA_NUM_IN_Q, NUM_IN_Q
    AREA_NUM_IN_Q += NUM_IN_Q * time_since_last_event

    # update area under server-busy indicator function
    global AREA_SERVER_STATUS, SERVER_STATUS
    AREA_SERVER_STATUS += SERVER_STATUS.value * time_since_last_event

    # update area under number of people in the system function
    global AREA_NUM_IN_SYSTEM
    AREA_NUM_IN_SYSTEM += (SERVER_STATUS.value + NUM_IN_Q) * time_since_last_event

def main():
    # initialize
    initialize()

    # run the simulation until all delays are observed
    while True:
        # determine the next event
        timing()
        
        # update time-average statistical accumulators
        update_time_avg_stats()

        # invoke the appropriate event function
        match NEXT_EVENT_TYPE:
            case 0:
                arrive()
            case 1:
                depart()
            case 2:
                print('exiting')
                # invoke the report generator and end the simulation
                report()
                break
    return

In [20]:
main()

exiting


Average delay in queue       0.487 minutes


Maximum delay in queue      4.164


Proportion of customers with delay > 1      0.173


Average total time in system       0.719 minutes


Maximum time in system      6.044


Average number in queue      0.509


Maximum number in queue          7


Average number in system      1.025


Server utilization           0.516


Time simulation ended      480.000 minutes
